In [70]:
import csv
import numpy as np
from os import listdir
from os.path import isfile, join
import os

'''
Usage:
1.put all song dirs into all_song_dir
2.each song dir should contain A.mid,B.mid
3.Result:create out_name.mid in each dir,which is fixed pitchbend midi file
'''

def fix_pitchbend(A_name,B_name,out_name):
    '''
    input : 2 csv file 
    A_name:file before modifying,
    B_name:file after modifying
    output: 1 csv file
    fix pitchbend
    '''
    fA = open(A_name, 'r')
    fB = open(B_name, 'r')
    wf = open(out_name, 'w')

    A_data = list(csv.reader(fA))
    B_data = list(csv.reader(fB))
    rows = len(A_data)
    # print(A_data[:][0])
    mod_flag = False
    del_tone = 0
    PBS = 2
    cent_per_tone = 8192/PBS

    for t in range(0,rows):
        if 'Note_on' in A_data[t][2]:
            if A_data[t][4]!=B_data[t][4]:
                del_tone = int(B_data[t][4])-int(A_data[t][4])
                mod_flag=True
                shift_cent = int(cent_per_tone*del_tone)
        elif 'Note_off' in A_data[t][2]:
            mod_flag=False
        elif mod_flag:
            B_data[t][4] = str(np.clip(int(B_data[t][4])+shift_cent ,0,16383))
        wf.write(','.join(B_data[t]))
        wf.write('\n')
    fA.close()
    fB.close()
    wf.close()
    

#================= start user params =================
all_song_dir = 'All_song'
A_name = 'A'
B_name = 'B'
out_name = 'B_fix'
#================== end user params   =================
A_mid_name = A_name + '.mid'
B_mid_name = B_name + '.mid'
for f in listdir(all_song_dir):
    song_path = str('./'+all_song_dir+'/'+f+'/')
    proj_files = [ff for ff in listdir(song_path) if isfile(join(song_path, ff))]
    for pf in  proj_files:
        if pf == A_mid_name or pf ==B_mid_name:
            csv_name = pf.replace('.mid','')+'.csv'
            os.system('Midicsv '+song_path+pf+' '+csv_name)
    print('procesing '+song_path)
    fix_pitchbend(A_name+'.csv',B_name+'.csv',out_name+'.csv')
    
    os.system('Csvmidi '+out_name+'.csv '+out_name+'.mid')
    if ~( os.system('move '+out_name +'.mid '+song_path)):
        print('move '+out_name +'.mid '+song_path)
   
    
os.system('del '+A_name+'.csv')
os.system('del '+B_name+'.csv')
os.system('del '+out_name+'.csv')

procesing ./All_song/Song1/
move B_fix.mid ./All_song/Song1/
procesing ./All_song/Song2/
move B_fix.mid ./All_song/Song2/


0